In [1]:
from PIL import Image, ImageDraw
import face_recognition
import os

In [8]:
def mouth_cropper(input_folder, output_folder):
    # Establish count for iterative file saving
    count = 0

    # Ensure the output folder exists or create it
    os.makedirs(output_folder, exist_ok=True)

    # For each file in the input folder
    for file in os.listdir(input_folder):
        # Load the image using face_recognition library
        image = face_recognition.load_image_file(os.path.join(input_folder, file))

        # Get face landmarks
        face_landmarks_list = face_recognition.face_landmarks(image)

        # Check if face landmarks are available
        if face_landmarks_list:
            # Extract mouth coordinates
            try:
                mouth = face_landmarks_list[0]['bottom_lip'] + face_landmarks_list[0]['top_lip']
            except KeyError:
                # Handle the case when mouth landmarks are not available
                continue

            # Calculate bounding box around the mouth
            x_coords = [coord[0] for coord in mouth]
            y_coords = [coord[1] for coord in mouth]

            left = min(x_coords)
            top = min(y_coords)
            right = max(x_coords)
            bottom = max(y_coords)

            # Crop the original image using mouth coordinates
            im = Image.open(os.path.join(input_folder, file))
            im = im.crop((left, top, right, bottom))

            # Resize image for input into your model
            im = im.resize((80, 80))

            # Save the cropped image to the output folder
            im.save(os.path.join(output_folder, file))

            # Increase count for iterative file saving
            count += 1

            # Print count every 200 photos to monitor progress
            if count % 200 == 0:
                print(count)

In [13]:
input_folder_path = '../data/archive/val/mouth/no_yawn/'
output_folder_path = '../data/archive/val/mouth-cropped/no_yawn/'
mouth_cropper(input_folder_path, output_folder_path)